In [1]:
import pandas as pd
import os
import tqdm
import pickle
import typing as T
from joblib import delayed, Parallel
FOLDER_ADDRESS = r'D:\Projects\stochastic-dominance'
os.chdir(FOLDER_ADDRESS)
from utils.prepare_data import clean_data
from dominance.dominance import StochasticDominance
price_data_address = os.path.join(FOLDER_ADDRESS,'data')
return_data = clean_data(price_data_address)

In [2]:
sd = StochasticDominance(2, 'Passive', 252, 'SPY')

In [3]:
START_DATE = pd.to_datetime('2021-01-01', utc=True)
END_DATE = pd.to_datetime('2023-08-01', utc=True)
N_JOBS = -1 # how many jobs to use for calculation of dominance if -1 means use all availabe threads

CALCULATION_FREQUENCY = 'm' # in what steps we want to calculate our dominance for example 
                            # in this case we use daily data and we want to backtest with monthly steps
dominace_calc_dates = pd.date_range(start=START_DATE, end=END_DATE, freq=CALCULATION_FREQUENCY)


def parallel_function(price_data: T.Dict,
                      date: pd.Timestamp):
    date_result = sd.get_dominance(price_data, 
                                   start_time=START_DATE, 
                                   end_time=date,
                                   names=['AAPL', 'GOOG', 'MSFT', 'SPY'],)
    return {date : date_result}


results = Parallel(n_jobs=N_JOBS)(delayed(parallel_function)(return_data, date) for date in dominace_calc_dates)

In [4]:
results

[{Timestamp('2021-01-31 00:00:00+0000', tz='UTC', freq='M'): {'AAPL': {'better_count': 0,
    'worst_count': 2,
    'name': 'AAPL',
    'better_names': [],
    'worst_names': ['GOOG', 'MSFT']},
   'GOOG': {'better_count': 1,
    'worst_count': 0,
    'name': 'GOOG',
    'better_names': ['AAPL'],
    'worst_names': []},
   'MSFT': {'better_count': 1,
    'worst_count': 0,
    'name': 'MSFT',
    'better_names': ['AAPL'],
    'worst_names': []},
   'SPY': {'better_count': 0,
    'worst_count': 0,
    'name': 'SPY',
    'better_names': [],
    'worst_names': []}}},
 {Timestamp('2021-02-28 00:00:00+0000', tz='UTC', freq='M'): {'AAPL': {'better_count': 0,
    'worst_count': 3,
    'name': 'AAPL',
    'better_names': [],
    'worst_names': ['GOOG', 'MSFT', 'SPY']},
   'GOOG': {'better_count': 1,
    'worst_count': 0,
    'name': 'GOOG',
    'better_names': ['AAPL'],
    'worst_names': []},
   'MSFT': {'better_count': 1,
    'worst_count': 0,
    'name': 'MSFT',
    'better_names': ['AAPL'],
